In [19]:
#SD_PSD_ML_manuscript.ipynb
#Arjun Balachandar Oct 2024
#This code takes the PSDs for each recording, and labels them as either SD or non-SD (often referred to as oneYear in the code), and then trains a machine learning model
# to predict the label of a new PSD, using leave-one-out cross validation.

#Directories and File Names
PATH = #change directory path accordingly
lfpDir = PATH + 'data_processed/'
clinVarFile = PATH + 'statusDystonicusDates_NEW.csv'
implantDateFile = PATH + 'patientSummary.csv'
optimalChannelsFile = PATH + 'AvailabilitySummary - with optimal channels - deidentified.xlsx' # Determined in consultation with Dr George Ibrahim
resultsDir = PATH + 'Analysis/'
resultsLFPPowersDir = resultsDir + 'LFP Powers/'
resultsPSDsDir = resultsDir + 'PSDs/'
resultsPSDPeaksDir = resultsDir + 'PSDPeaks/'
resultsFilename = 'powerChangesAfterCuringStatusDystonicus.csv'
resultsPSDFilename = 'PSDsAfterCuringStatusDystonicus.csv'
resultsCohDir = resultsDir + 'Coherence/'
resultsCohFilename = 'coherenceAfterCuringStatusDystonicus.csv'
demographicsDir = PATH + 'Demographics/'
demographicsFile = demographicsDir + 'MulticentrePediatric-Demographics and clinical data (raw)_ArjunEdit.csv'
moredemographicsDir = PATH + 'Demographics/Additional Metrics/' #additional clinical metrics
moredemographicsBasicInfo = moredemographicsDir + 'MulticentrePediatric-ClinicalData Dystonia Labelled.csv'
moredemographicsFile_baseline = moredemographicsDir + 'MulticentrePediatric-BaselineDystonia.csv'
moredemographicsFile_6m = moredemographicsDir + 'MulticentrePediatric-6mDystonia.csv'
moredemographicsFile_1y = moredemographicsDir + 'MulticentrePediatric-1yDystonia.csv'
moredemographicsFile_2y = moredemographicsDir + 'MulticentrePediatric-2yDystonia.csv'
moredemographicsFile_3y = moredemographicsDir + 'MulticentrePediatric-3yDystonia.csv'

#Frequency bands
betaLim = [12.5, 30] # In Hertz, define the beta band
thetaLim = [3, 7] # 3.5-7 as per Mark, Was [4,7] before 31 Jan
deltaLim = [1,3]
alphaLim = [7,12.5]
gammaLim = [30,60]
lowbetaLim = [12.5, 20]
highbetaLim = [20, 30]
lowgammaLim = [30, 40]
highgammaLim = [40, 60]
channelOverride = False # True # Should be false. Set to true if you want to set the channel manually (not recommended).
z_score_norm = False #Normalize the LFP values by z-scoring normalization if True
normalize_PSD = True #Normalize the PSD values by dividing by sum of total power of PSD

# Initializations
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from scipy import signal
from scipy.signal import welch
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import zscore
import re #regular expression
import neurodsp
from neurodsp.utils.norm import normalize_sig
from neurodsp.utils import create_times
from neurodsp.plts.time_series import plot_time_series, plot_instantaneous_measure
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, FunctionTransformer
import itertools
import seaborn as sns
import statsmodels.api as sm
from statsmodels.robust.norms import HuberT
from scipy.stats import mannwhitneyu
from scipy.stats import kruskal
import statsmodels.formula.api as smf
import scikit_posthocs as sp
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import TheilSenRegressor

#Machine Learning
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn_lvq import GrmlvqModel
from sklvq import LGMLVQ, GMLVQ
from sklearn_lvq.utils import plot2d
import lightgbm as lgb
import shap

diff = lambda x, y: y - x

# Load and process the dates
clinVars_df = pd.read_csv(clinVarFile, delimiter=',')
implantDates_df = pd.read_csv(implantDateFile, delimiter=',')
patList = clinVars_df['Subject ID'].values
patList

array(['DBS003', 'DBS005', 'DBS013', 'DBS014', 'DBS018', 'DBS021',
       'DBS036', 'DBS047', 'DBS050', 'DBS039'], dtype=object)

In [49]:
#LOOCV - suboptimal method to LOPOCV (see below)

# Define the optimal channels for each patient
def get_hemisphere_channels(patient, channel):
    # Extract hemisphere from the channel name
    if channel.endswith('_RIGHT'):
        hemisphere_channel = 'RIGHT'
    elif channel.endswith('_LEFT'):
        hemisphere_channel = 'LEFT'
    else:
        return None  # Return None if it's not a valid channel
    
    # Return optimal channels based on the patient and hemisphere
    if patient == 'DBS036' and hemisphere_channel == 'LEFT':
        return ['ZERO_THREE']
    if patient == 'DBS009' and hemisphere_channel == 'RIGHT':
        return ['ZERO_THREE']
    if patient == 'DBS014' and hemisphere_channel == 'LEFT':
        return ['ZERO_TWO']
    if patient == 'DBS013' and hemisphere_channel == 'RIGHT':
        return ['ONE_THREE']
    if patient == 'DBS021' and hemisphere_channel == 'LEFT':
        return ['ZERO_TWO']
    if patient == 'DBS021' and hemisphere_channel == 'RIGHT':
        return ['ZERO_TWO']
    if patient == 'DBS005' and hemisphere_channel == 'LEFT':
        return ['ONE_THREE']
    if patient == 'DBS005' and hemisphere_channel == 'RIGHT':
        return ['TWO_THREE']
    if patient == 'DBS047' and hemisphere_channel == 'LEFT':
        return ['ZERO_TWO']
    if patient == 'DBS047' and hemisphere_channel == 'RIGHT':
        return ['ZERO_TWO']
    if patient == 'DBS050' and hemisphere_channel == 'LEFT':
        return ['ZERO_TWO']
    if patient == 'DBS039' and hemisphere_channel == 'LEFT':
        return ['ONE_THREE']
    return ['ONE_THREE']

def get_optimal_hemisphere(patient): #function returns the optimal hemisphere for a patient, so only PSD from the optimal hemisphere will be used for ML
    if patient == 'DBS003':
        return 'RIGHT'
    if patient == 'DBS005':
        return 'RIGHT'
    if patient == 'DBS013':
        return 'RIGHT'
    if patient == 'DBS014':
        return 'LEFT'
    if patient == 'DBS018':
        return 'RIGHT'
    if patient == 'DBS021':
        return 'RIGHT'
    if patient == 'DBS036':
        return 'RIGHT'
    if patient == 'DBS039':
        return 'LEFT'
    if patient == 'DBS047':
        return 'RIGHT'
    if patient == 'DBS050':
        return 'LEFT'
    return 'RIGHT'

PSD_df = pd.read_csv(resultsDir + resultsPSDFilename)
PSD_df['RecordingType'] = PSD_df['RecordingType'].replace('CT', 'CT (stim on)')
df = pd.read_csv(resultsDir + resultsFilename)
df['RecordingType'] = df['RecordingType'].replace('CT', 'CT (stim on)')

patList = PSD_df['Patient'].unique() #patList = ['DBS003'], hemisphere = 'RIGHT'

PSD_IDpowers = PSD_df.drop(['Implant date','StatusDysStart','Discharge','RecordingStartTime','RecordingType','Channel'], axis=1)

aggregated_shap_values = np.zeros(len(freqs))  # Sum SHAP values across folds


#USE ALL FREQUENCIES AS FEATURES
freqs = [col for col in PSD_df.columns if re.search(r'\d', col)]
freqs_np = np.array(freqs)

#Use column 'IntervalType' to determine the label of the recording, and the columns in freqs to determine the features
#Create a new dataframe with the features and labels
X = []
y = []

for index, row in PSD_df.iterrows():
    optimal_hemisphere_channels = get_hemisphere_channels(row['Patient'], row['Channel'])
    if row['Channel'].endswith('_RIGHT'):
        hemisphere = 'RIGHT'
        channel = row['Channel'].replace('_RIGHT', '')
    elif row['Channel'].endswith('_LEFT'):
        hemisphere = 'LEFT'
        channel = row['Channel'].replace('_LEFT', '')
    #print(row['Patient'], row['Channel'], optimal_hemisphere_channels)
    if channel in optimal_hemisphere_channels and hemisphere == get_optimal_hemisphere(row['Patient']): #only use PSDs from the optimal hemisphere and contact
        # Now check whether to add to train or test set
        X.append(row[freqs].values)
        if row['IntervalType'] == 'status':
            y.append(1)
        elif row['IntervalType'] == 'oneYear':
            y.append(0)
        else:
            print('Error: IntervalType is not SD or oneYear')
            break

X = np.array(X)
y = np.array(y)

print(len(y))

#Train random forect classifier to predict the label of a new PSD, using leave-one-out cross validation.

loo = LeaveOneOut()
y_pred = []
y_true = []
for train_index, test_index in loo.split(X):
    #print the progress of the training
    print('Training:', len(y_true) + 1, 'of', len(y) + 1)
    print(len(X[train_index]), len(X[test_index]))
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf = RandomForestClassifier()

    clf.fit(np.array(X_train), np.array(y_train))
    y_pred.append(clf.predict(X_test))
    y_true.append(y_test)

    explainer = shap.TreeExplainer(clf)
    shap_values = explainer.shap_values(X_train)

    if isinstance(shap_values, list):  # Binary classification
        # Combine SHAP values across samples
        shap_values_combined = np.mean(np.abs(shap_values), axis=0)  # Shape: (n_samples, n_features)
        print(shap_values_combined.shape)
        # Compute mean absolute SHAP values for each feature
        mean_shap_values_per_class = np.mean(shap_values_combined, axis=0)  # Shape: (n_features, 2)

        # Take the first column, as both are equal
        mean_shap_values = mean_shap_values_per_class[:, 0]  # Shape: (n_features,)
    else:
        mean_shap_values = np.abs(shap_values).mean(axis=0)  # Shape: (n_features,)


    # Accumulate SHAP values
    aggregated_shap_values += mean_shap_values[:,0]  # Now properly reduced to (129,)
    fold_count += 1

# Average SHAP values across folds
aggregated_shap_values /= fold_count

# Create feature importance DataFrame
feature_importance_df = pd.DataFrame({
    'Feature': freqs,
    'Mean SHAP Value': aggregated_shap_values
}).sort_values(by='Mean SHAP Value', ascending=False)

y_pred = np.array(y_pred)
y_true = np.array(y_true)
y_pred = y_pred.flatten()
y_true = y_true.flatten()
print('Accuracy:', accuracy_score(y_true, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_true, y_pred))
print('Classification Report:')
print(classification_report(y_true, y_pred))
roc_auc = roc_auc_score(y_true, y_pred)
print('ROC AUC:', roc_auc)

# Define the beta-band range
betaLim = [12.5, 30]

# Extract the top 30 features
top_features = feature_importance_df.head(20)
import re
top_features['FeatureValue'] = top_features['Feature'].apply(
    lambda x: float(re.findall(r"[-+]?\d*\.\d+|\d+", x)[0]) if re.search(r"[-+]?\d*\.\d+|\d+", x) else None
)

# Count the number of features within the beta-band range
num_beta_band_features = top_features['FeatureValue'].apply(
    lambda x: betaLim[0] <= x <= betaLim[1] if x is not None else False
).sum()

print(f"Number of features in the beta-band range: {num_beta_band_features}")
print(feature_importance_df.head(20))


<>:80: SyntaxWarning: invalid escape sequence '\d'
<>:80: SyntaxWarning: invalid escape sequence '\d'
/var/folders/qr/d2wxfs456fsf5lgfshrqbsmw0000gn/T/ipykernel_10585/224699477.py:80: SyntaxWarning: invalid escape sequence '\d'
  '''


183
Training: 1 of 184
182 1
Training: 2 of 184
182 1
Training: 3 of 184
182 1
Training: 4 of 184
182 1
Training: 5 of 184
182 1
Training: 6 of 184
182 1
Training: 7 of 184
182 1
Training: 8 of 184
182 1
Training: 9 of 184
182 1
Training: 10 of 184
182 1
Training: 11 of 184
182 1
Training: 12 of 184
182 1
Training: 13 of 184
182 1
Training: 14 of 184
182 1
Training: 15 of 184
182 1
Training: 16 of 184
182 1
Training: 17 of 184
182 1
Training: 18 of 184
182 1
Training: 19 of 184
182 1
Training: 20 of 184
182 1
Training: 21 of 184
182 1
Training: 22 of 184
182 1
Training: 23 of 184
182 1
Training: 24 of 184
182 1
Training: 25 of 184
182 1
Training: 26 of 184
182 1
Training: 27 of 184
182 1
Training: 28 of 184
182 1
Training: 29 of 184
182 1
Training: 30 of 184
182 1
Training: 31 of 184
182 1
Training: 32 of 184
182 1
Training: 33 of 184
182 1
Training: 34 of 184
182 1
Training: 35 of 184
182 1
Training: 36 of 184
182 1
Training: 37 of 184
182 1
Training: 38 of 184
182 1
Training: 39 of 1

/var/folders/qr/d2wxfs456fsf5lgfshrqbsmw0000gn/T/ipykernel_10585/224699477.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_features['FeatureValue'] = top_features['Feature'].apply(


In [34]:
#LOPOCV = HAVE SEPARATE TRAINING AND TESTING SET WITH DIFFERENT PATIENTS
#Load the PSDs for all recording, and label them as either SD or non-SD

# Define the optimal channels for each patient
def get_hemisphere_channels(patient, channel):
    if channel.endswith('_RIGHT'):
        hemisphere_channel = 'RIGHT'
    elif channel.endswith('_LEFT'):
        hemisphere_channel = 'LEFT'
    else:
        return None  # Return None if it's not a valid channel
    
    # Return optimal channels based on the patient and hemisphere
    if patient == 'DBS036' and hemisphere_channel == 'LEFT':
        return ['ZERO_THREE']
    if patient == 'DBS009' and hemisphere_channel == 'RIGHT':
        return ['ZERO_THREE']
    if patient == 'DBS014' and hemisphere_channel == 'LEFT':
        return ['ZERO_TWO']
    if patient == 'DBS013' and hemisphere_channel == 'RIGHT':
        return ['ONE_THREE']
    if patient == 'DBS021' and hemisphere_channel == 'LEFT':
        return ['ZERO_TWO']
    if patient == 'DBS021' and hemisphere_channel == 'RIGHT':
        return ['ZERO_TWO']
    if patient == 'DBS005' and hemisphere_channel == 'LEFT':
        return ['ONE_THREE']
    if patient == 'DBS005' and hemisphere_channel == 'RIGHT':
        return ['TWO_THREE']
    if patient == 'DBS047' and hemisphere_channel == 'LEFT':
        return ['ZERO_TWO']
    if patient == 'DBS047' and hemisphere_channel == 'RIGHT':
        return ['ZERO_TWO']
    if patient == 'DBS050' and hemisphere_channel == 'LEFT':
        return ['ZERO_TWO']
    if patient == 'DBS039' and hemisphere_channel == 'LEFT':
        return ['ONE_THREE']
    return ['ONE_THREE']

def get_optimal_hemisphere(patient): #function returns the optimal hemisphere for a patient, so only PSD from the optimal hemisphere will be used for ML
    if patient == 'DBS003':
        return 'RIGHT'
    if patient == 'DBS005':
        return 'RIGHT'
    if patient == 'DBS013':
        return 'RIGHT'
    if patient == 'DBS014':
        return 'LEFT'
    if patient == 'DBS018':
        return 'RIGHT'
    if patient == 'DBS021':
        return 'RIGHT'
    if patient == 'DBS036':
        return 'RIGHT'
    if patient == 'DBS039':
        return 'LEFT'
    if patient == 'DBS047':
        return 'RIGHT'
    if patient == 'DBS050':
        return 'LEFT'
    return 'RIGHT'

PSD_df = pd.read_csv(resultsDir + resultsPSDFilename)
PSD_df['RecordingType'] = PSD_df['RecordingType'].replace('CT', 'CT (stim on)')
df = pd.read_csv(resultsDir + resultsFilename)

patList = PSD_df['Patient'].unique()

test_patList_combos = []
no_postSD_pats = ['DBS005', 'DBS014', 'DBS018']  # no non-SD to test foraccuracy of classification of non-SD on
test_patList_combos = []

for perm in itertools.combinations(patList, 1): #choose every combination of 1 patients as test set, and 9 as training
    test_patList_combos.append(perm)

    
PSD_IDpowers = PSD_df.drop(['Implant date','StatusDysStart','Discharge','RecordingStartTime','RecordingType','Channel'], axis=1)

#USE ALL FREQUENCIES AS FEATURES
freqs = [col for col in PSD_df.columns if re.search(r'\d', col)]
freqs_np = np.array(freqs)

#Use column 'IntervalType' to determine the label of the recording, and the columns in freqs to determine the features
#Create a new dataframe with the features and labels
X_train = []
y_train = []
X_test = []
y_test = []

y_pred = []
y_true = []

for test_patList in test_patList_combos: #iterate through combinations test sets
    print('Test Patients:', test_patList)
    X_train = []
    y_train = []
    X_test = []
    y_test = []

    for index, row in PSD_df.iterrows():
        optimal_hemisphere_channels = get_hemisphere_channels(row['Patient'], row['Channel'])
        if row['Channel'].endswith('_RIGHT'):
            hemisphere = 'RIGHT'
            channel = row['Channel'].replace('_RIGHT', '')
        elif row['Channel'].endswith('_LEFT'):
            hemisphere = 'LEFT'
            channel = row['Channel'].replace('_LEFT', '')
        if channel in optimal_hemisphere_channels and hemisphere == get_optimal_hemisphere(row['Patient']): #only use PSDs from the optimal hemisphere and contact
            # Now check whether to add to train or test set
            if row['Patient'] in test_patList:
                X_test.append(row[freqs].values)
                if row['IntervalType'] == 'status':
                    y_test.append(1)
                elif row['IntervalType'] == 'oneYear':
                    y_test.append(0)
                else:
                    print('Error: IntervalType is not SD or oneYear')
                    break
            else:
                X_train.append(row[freqs].values)
                if row['IntervalType'] == 'status':
                    y_train.append(1)
                elif row['IntervalType'] == 'oneYear':
                    y_train.append(0)
                else:
                    print('Error: IntervalType is not SD or oneYear')
                    break
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)

    clf = RandomForestClassifier()

    clf.fit(np.array(X_train), np.array(y_train))
    y_pred.extend(clf.predict(X_test))
    y_true.extend(y_test)

y_pred = np.array(y_pred)
y_true = np.array(y_true)

y_pred = y_pred.flatten()
y_true = y_true.flatten()
print('Accuracy:', accuracy_score(y_true, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_true, y_pred))
print('Classification Report:')
print(classification_report(y_true, y_pred))
roc_auc = roc_auc_score(y_true, y_pred)
print('ROC AUC:', roc_auc)


<>:92: SyntaxWarning: invalid escape sequence '\d'
<>:92: SyntaxWarning: invalid escape sequence '\d'
/var/folders/qr/d2wxfs456fsf5lgfshrqbsmw0000gn/T/ipykernel_10585/3816093064.py:92: SyntaxWarning: invalid escape sequence '\d'
  '''


Test Patients: ('DBS003',)
Test Patients: ('DBS005',)
Test Patients: ('DBS013',)
Test Patients: ('DBS014',)
Test Patients: ('DBS018',)
Test Patients: ('DBS021',)
Test Patients: ('DBS036',)
Test Patients: ('DBS047',)
Test Patients: ('DBS050',)
Test Patients: ('DBS039',)
Accuracy: 0.7486338797814208
Confusion Matrix:
[[65 34]
 [12 72]]
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.66      0.74        99
           1       0.68      0.86      0.76        84

    accuracy                           0.75       183
   macro avg       0.76      0.76      0.75       183
weighted avg       0.77      0.75      0.75       183

ROC AUC: 0.7568542568542569


"\nloo = LeaveOneOut()\ny_pred = []\ny_true = []\nfor train_index, test_index in loo.split(X):\n    #print the progress of the training\n    print('Training:', len(y_true) + 1, 'of', len(y) + 1)\n    X_train, X_test = X[train_index], X[test_index]\n    y_train, y_test = y[train_index], y[test_index]\n    clf = svm.SVC()\n    #clf = GMLVQ()\n    #clf = GrmlvqModel()\n    #clf = RandomForestClassifier()\n    #clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)\n    clf = XGBClassifier()\n    clf.fit(np.array(X_train), np.array(y_train))\n    y_pred.append(clf.predict(X_test))\n    y_true.append(y_test)\n"

In [35]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
import shap

# Define functions for optimal channels and hemispheres
def get_hemisphere_channels(patient, channel):
    if channel.endswith('_RIGHT'):
        hemisphere_channel = 'RIGHT'
    elif channel.endswith('_LEFT'):
        hemisphere_channel = 'LEFT'
    else:
        return None

    if patient == 'DBS036' and hemisphere_channel == 'LEFT':
        return ['ZERO_THREE']
    if patient == 'DBS009' and hemisphere_channel == 'RIGHT':
        return ['ZERO_THREE']
    if patient == 'DBS014' and hemisphere_channel == 'LEFT':
        return ['ZERO_TWO']
    if patient == 'DBS013' and hemisphere_channel == 'RIGHT':
        return ['ONE_THREE']
    if patient == 'DBS021' and hemisphere_channel == 'LEFT':
        return ['ZERO_TWO']
    if patient == 'DBS021' and hemisphere_channel == 'RIGHT':
        return ['ZERO_TWO']
    if patient == 'DBS005' and hemisphere_channel == 'LEFT':
        return ['ONE_THREE']
    if patient == 'DBS005' and hemisphere_channel == 'RIGHT':
        return ['TWO_THREE']
    if patient == 'DBS047' and hemisphere_channel == 'LEFT':
        return ['ZERO_TWO']
    if patient == 'DBS047' and hemisphere_channel == 'RIGHT':
        return ['ZERO_TWO']
    if patient == 'DBS050' and hemisphere_channel == 'LEFT':
        return ['ZERO_TWO']
    if patient == 'DBS039' and hemisphere_channel == 'LEFT':
        return ['ONE_THREE']
    return ['ONE_THREE']

def get_optimal_hemisphere(patient):
    if patient == 'DBS003':
        return 'RIGHT'
    if patient == 'DBS005':
        return 'RIGHT'
    if patient == 'DBS013':
        return 'RIGHT'
    if patient == 'DBS014':
        return 'LEFT'
    if patient == 'DBS018':
        return 'RIGHT'
    if patient == 'DBS021':
        return 'RIGHT'
    if patient == 'DBS036':
        return 'RIGHT'
    if patient == 'DBS039':
        return 'LEFT'
    if patient == 'DBS047':
        return 'RIGHT'
    if patient == 'DBS050':
        return 'LEFT'
    return 'RIGHT'

# Load data
PSD_df = pd.read_csv(resultsDir + resultsPSDFilename)
patList = PSD_df['Patient'].unique()

# Prepare test patient combinations
test_patList_combos = []
no_postSD_pats = ['DBS005', 'DBS014', 'DBS018']
for perm in itertools.combinations(patList, 1):
#    if not any(pat in no_postSD_pats for pat in perm):
    test_patList_combos.append(perm)

freqs = [col for col in PSD_df.columns if re.search(r'\d', col)]
freqs_np = np.array(freqs)

# Initialize variables for results
y_pred = []
y_true = []
aggregated_shap_values = np.zeros(len(freqs))  # Sum SHAP values across folds
fold_count = 0

# Loop through test patient combinations
for test_patList in test_patList_combos:
    print('Test Patients:', test_patList)
    X_train, y_train, X_test, y_test = [], [], [], []

    for index, row in PSD_df.iterrows():
        optimal_hemisphere_channels = get_hemisphere_channels(row['Patient'], row['Channel'])
        if row['Channel'].endswith('_RIGHT'):
            hemisphere = 'RIGHT'
            channel = row['Channel'].replace('_RIGHT', '')
        elif row['Channel'].endswith('_LEFT'):
            hemisphere = 'LEFT'
            channel = row['Channel'].replace('_LEFT', '')

        if channel in optimal_hemisphere_channels and hemisphere == get_optimal_hemisphere(row['Patient']):
            if row['Patient'] in test_patList:
                X_test.append(row[freqs].values)
                y_test.append(1 if row['IntervalType'] == 'status' else 0)
            else:
                X_train.append(row[freqs].values)
                y_train.append(1 if row['IntervalType'] == 'status' else 0)

    X_train, y_train = np.array(X_train), np.array(y_train)
    X_test, y_test = np.array(X_test), np.array(y_test)

    # Train Random Forest model
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    y_pred.extend(clf.predict(X_test))
    y_true.extend(y_test)

    # Compute SHAP values
    explainer = shap.TreeExplainer(clf)
    shap_values = explainer.shap_values(X_train)

    #set shap_values to be the absolute value of the first row of shap_values

    #shap_values = shap_values[:,0]
    #print(shap_values.shape)

    if isinstance(shap_values, list):  # Binary classification
        # Combine SHAP values across samples
        shap_values_combined = np.mean(np.abs(shap_values), axis=0)  # Shape: (n_samples, n_features)
        print(shap_values_combined.shape)
        # Compute mean absolute SHAP values for each feature
        mean_shap_values_per_class = np.mean(shap_values_combined, axis=0)  # Shape: (n_features, 2)

        # Take the first column, as both are equal
        mean_shap_values = mean_shap_values_per_class[:, 0]  # Shape: (n_features,)
    else:
        mean_shap_values = np.abs(shap_values).mean(axis=0)  # Shape: (n_features,)


    # Accumulate SHAP values
    aggregated_shap_values += mean_shap_values[:,0]  # Now properly reduced to (129,)
    fold_count += 1

# Average SHAP values across folds
aggregated_shap_values /= fold_count

# Create feature importance DataFrame
feature_importance_df = pd.DataFrame({
    'Feature': freqs,
    'Mean SHAP Value': aggregated_shap_values
}).sort_values(by='Mean SHAP Value', ascending=False)

# Display results
print('Accuracy:', accuracy_score(y_true, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_true, y_pred))
print('Classification Report:')
print(classification_report(y_true, y_pred))
print('ROC AUC:', roc_auc_score(y_true, y_pred))

# Output feature importance
print(feature_importance_df.head(30))



Test Patients: ('DBS003',)
Test Patients: ('DBS005',)
Test Patients: ('DBS013',)
Test Patients: ('DBS014',)
Test Patients: ('DBS018',)
Test Patients: ('DBS021',)
Test Patients: ('DBS036',)
Test Patients: ('DBS047',)
Test Patients: ('DBS050',)
Test Patients: ('DBS039',)
Accuracy: 0.7431693989071039
Confusion Matrix:
[[63 36]
 [11 73]]
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.64      0.73        99
           1       0.67      0.87      0.76        84

    accuracy                           0.74       183
   macro avg       0.76      0.75      0.74       183
weighted avg       0.77      0.74      0.74       183

ROC AUC: 0.7527056277056278
       Feature  Mean SHAP Value
34   33.203125         0.026156
31  30.2734375         0.019699
36    35.15625         0.019234
37  36.1328125         0.016261
18   17.578125         0.014353
35  34.1796875         0.012451
32       31.25         0.011878
30   29.296875         0.01107